# Test Driven Development

- write better code

In [ ]:
!pip install -q numpy pandas

import numpy as np
import pandas as pd

## `assert()` 

Asserts 
- are **low overhead**
- help the reader understand code

In [ ]:
assert True

assert 1 == 1

In [ ]:
assert False
assert 0 == 1

Common asserts in machine learning are to check the shapes of your training and test data:

In [ ]:
x_train = np.array([10, 20, 30]).reshape(3, 1)
x_test = np.array([40, 50]).reshape(2, 1)

y_train = np.array( ['a', 'b', 'c']).reshape(3, 1)
y_test = np.array( ['d', 'e']).reshape(2, 1)

assert x_train.shape[0] == y_train.shape[0]
assert x_test.shape[1] == x_train.shape[1]

## `.any()` and `.all()`

These reduce across iterables:

In [ ]:
any((False, False))

In [ ]:
any((True, False))

In [ ]:
all((True, False))

In [ ]:
all((True, True))

## `np.testing`

Upgrade on using Python builtins - useful when working with arrays

[Documentation](https://docs.scipy.org/doc/numpy/reference/routines.testing.html)

In [ ]:
np.testing.assert_array_equal?

## Exercise

Write a one passing case of using `np.testing.assert_array_equal`
- you supply two arrays (data & expected)

In [ ]:
#  change these to numpy arrays that will pass
data = None
expected = None

assert np.testing.assert_array_equal(data, expected)

And one failing case:

In [ ]:
#  change these to numpy arrays that will fail
data = None
expected = None

assert not np.testing.assert_array_equal(data, expected)

## Test driven development

The first step in developing code in a TDD style is to 
- write the function skeleton
- write the test
- check the test fails

Below I show TTD code for doing **one-hot encoding**:

In [ ]:
def one_hot(data):
    pass


def test_one_hot():
    data = ['left', 'left', 'right', 'straight']
    
    encoded = one_hot(data)
    
    #  check we have the correct number of rows
    assert encoded.shape[0] == len(data)
    
    #  check columns are in the correct order (alphabetical)
    assert (encoded.columns == ['left', 'right', 'straight']).all()
    
    #  check only one category in each row
    assert encoded.sum(axis=1).all() == 1
    
    #  numpy testing is very handy for comparing arrays
    np.testing.assert_array_equal(encoded.loc[:, 'left'], [1, 1, 0, 0])
    np.testing.assert_array_equal(encoded.loc[:, 'right'], [0, 0, 1, 0])
    np.testing.assert_array_equal(encoded.loc[:, 'straight'], [0, 0, 0, 1])
    
test_one_hot()

Note how before we have written any functional code, we are already thinking about
- what order the columns should be in (alphabetical)
- that encoded should return a Pandas DataFrame

Also note how we (as the reader) can understand the intention of the function - this is executable documentation.

Lets write our function:

In [ ]:
def one_hot(data):
    columns = sorted(set(data))

    values = np.zeros((len(data), len(columns)))

    for row, d in enumerate(data):
        col = columns.index(d)
        values[row, col] = 1
        
    return pd.DataFrame(values, columns=columns)

test_one_hot()

Now lets see if our function generalizes:

In [ ]:
one_hot(['cat', 'dog', 'fish', 'fish', 'dog'])

## Normalization

An alternative to stardization

Scaling to between zero and one

$$ y = \frac{x - x_{min}}{x_{max} - x_{min}} $$

## Exercise

Now follow the same TDD style to write a test & function to normalize a 2D array

In [ ]:
def normalize(arr):
    pass

def test_normalize():
    pass

You can see the answers by importing them and using iPython's `??` to print the source code:

In [ ]:
from answers import test_normalize

test_normalize()

#test_normalize??

##  Standardization

An alternative to normalization

Removing the mean and scaling by the standard deviation

$$ y = \frac{x-\mu}{\sigma} $$

## Exercise

Write a test & function to standardize a 3D array
- standardize across the third dimension
- example in ML = sequential data of shape=(batch_size, num_timesteps, features)

You can see the answers by importing them and using iPython's `??` to print the source code:

In [ ]:
from answers import test_standardizer

test_standardizer()

#test_stanardizer??

## Practical - decompose the code so you can test it

A common task that
- you can do to help
- you will need to do to improve a colleagues code

You can write at least three functions from the below.  You should also remove any **magic numbers** (hard coded inputs).  There are bugs in the code below.

In [ ]:
np.random.seed(42)

#  20 samples, 4 columns
data = np.random.random((20, 4))

#  feature engineering - normalize
mins = np.min(data, axis=0)
maxs = np.max(data, axis=0)
data = (data - mins) / (maxs - mins)

#  feature engineering - split into x & y
x = data[:, :-1]
y = data[:, -1]

#  feature engineering - one hot encoding days
days = np.array([np.arange(0, 7) for _ in range(int(1 + data.shape[0] / 7))]).reshape(-1, 1)
days = days[:20, :]
days.shape

one_hot = np.zeros((len(days), 7))

columns = np.arange(0, 7).tolist()
for row, d in enumerate(days):
    one_hot[row, columns.index(d)] = 1
    
x = np.concatenate([x, data, one_hot], axis=1)

x_train = x[:5, :]
y_train = x[:5, :]

x_test = x[:5, :]
y_test = x[:5, :]